#### Download the Pascal-VOC dataset from [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit)

In [8]:
"""%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../"""

'%%shell\n\n# Download TorchVision repo to use some files from\n# references/detection\ngit clone https://github.com/pytorch/vision.git\ncd vision\ngit checkout v0.8.2\n\ncp references/detection/utils.py ../\ncp references/detection/transforms.py ../\ncp references/detection/coco_eval.py ../\ncp references/detection/engine.py ../\ncp references/detection/coco_utils.py ../'

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [11]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

## Section 1: Load Pascal-VOC dataset


In [12]:
import os
import cv2
import glob
import numpy as np
from xml.etree import ElementTree
train_dir = "/content/drive/MyDrive/Colab Notebooks/question3 /data/VOCdevkit/VOC2012/JPEGImages"
test_dir = 'your_dir_to_test_set'
from PIL import Image
from torchvision.transforms import Compose, ToTensor
class PascalVOCDataset(torch.utils.data.Dataset):
  def __init__(self, train_dir, width, height, transforms=None):
    self.transforms = transforms
    self.train_dir = train_dir
    self.height = height
    self.width = width
  
  # List of all classes in Pascal-VOC dataset
    self.classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
                    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
                    'dog', 'horse', 'motorbike', 'person', 'pottedplant',
                    'sheep', 'sofa', 'train', 'tvmonitor']       
  # Get list of all JPG files in train_dir
    self.jpg_files = glob.glob(os.path.join(train_dir, '*.jpg'))
    self.jpg_files.sort()
  #TODO: Declare a list all classes in Pascal-VOC dataset e.g. [aeroplane, bicycle, boat, bus, car]

  def __getitem__(self, idx):
    #TODO: Converting image to correct size
    #img_res = cv2.resize(...)
    #TODO Extract the necessary information from xml files - bounding box coordinates, label, area, image_id
    #target = {"bndboxes": [], "labels" : [], "area" : [], "image_id" : []} 
    #return img_res, target
    # Read image
    img_path = self.jpg_files[idx]
    #print("img",img_path)
    img = cv2.imread(img_path)
    # Convert NumPy array to PIL image
    img = Image.fromarray(img)

    # Resize PIL image
    img = img.resize((self.width, self.height))
   


    # Resize image
    #img_res = cv2.resize(img, (self.width, self.height))

    # Extract information from XML file
    xml_path = os.path.splitext(img_path)[0] + '.xml'
    #print("xml",xml_path)
    tree = ElementTree.parse(xml_path)
    root = tree.getroot()

    boxes = []
    labels = []
    areas = []
    image_id = []
    for obj in root.findall('object'):
        label = obj.find('name').text
 
        if label in self.classes:
          labels.append(self.classes.index(label))
        else:
          print("Invalid class label:", label)

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])

        area = (xmax - xmin) * (ymax - ymin)
        areas.append(area)
        image_id.append(idx)
    target = {"boxes": boxes, "labels": labels, "area": areas, "image_id": image_id}
    

    if self.transforms:
            img = self.transforms(img)
    
    #print("araf",len(target))
    return img, target



  def __len__(self):
      return len(self.jpg_files) #returns the number of images in the training directory.
      

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    targets = [{k: torch.tensor(v) for k, v in t.items()} for t in targets]
    return images, targets



## Section 2: Define Pre-trained Object Detection Model

In [13]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#from yolov5.models import YOLOv5

#import other models as well

def getPretrainedModel(num_classes):
  #TODO: load a pre-trained model and replace the pre-trained head with a new one.
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  
  # Replace pre-trained head
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  return model 

## Section 3: Prepare Dataset and Train the model



In [14]:
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import random_split
from torchvision.transforms import ToTensor

data_dir = "/content/drive/MyDrive/Colab Notebooks/question3 /data/VOCdevkit/VOC2012/JPEGImages"

transforms = transforms.Compose([
    transforms.Resize((520, 480)),
    transforms.ToTensor(),
])

dataset = PascalVOCDataset(data_dir, 520, 480, transforms=transforms)
torch.manual_seed(1)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print("dataset",len(dataset))


trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=2,collate_fn=collate_fn)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=2,collate_fn=collate_fn)


dataset 17130


In [15]:
 # these are the helper libraries imported.

from engine import train_one_epoch, evaluate

In [16]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 20

model = getPretrainedModel(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

torch.cuda.empty_cache()
lr = 0.001
num_epochs = 1
#batch_size = 4
optimizer = torch.optim.SGD(params, lr=lr, momentum=0.9)
for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, trainloader, device, epoch, print_freq=10)

    # update the learning rate
    lr.step()

    # evaluate on the test dataset
    evaluate(model, testloader, device=device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 57.8MB/s]


Epoch: [0]  [   0/1371]  eta: 5:45:02  lr: 0.000002  loss: 3.2523 (3.2523)  loss_classifier: 3.0936 (3.0936)  loss_box_reg: 0.1084 (0.1084)  loss_objectness: 0.0396 (0.0396)  loss_rpn_box_reg: 0.0106 (0.0106)  time: 15.1005  data: 4.6480  max mem: 6907
Epoch: [0]  [  10/1371]  eta: 1:02:28  lr: 0.000012  loss: 3.3779 (3.3824)  loss_classifier: 3.1187 (3.1271)  loss_box_reg: 0.1483 (0.1477)  loss_objectness: 0.0693 (0.0942)  loss_rpn_box_reg: 0.0126 (0.0134)  time: 2.7540  data: 1.4574  max mem: 7067
Epoch: [0]  [  20/1371]  eta: 7:54:06  lr: 0.000022  loss: 3.3468 (3.3536)  loss_classifier: 3.0457 (3.0157)  loss_box_reg: 0.1443 (0.1474)  loss_objectness: 0.1228 (0.1703)  loss_rpn_box_reg: 0.0145 (0.0203)  time: 21.3538  data: 20.9675  max mem: 7067
Epoch: [0]  [  30/1371]  eta: 5:21:35  lr: 0.000032  loss: 3.0015 (3.1592)  loss_classifier: 2.7067 (2.8131)  loss_box_reg: 0.1443 (0.1515)  loss_objectness: 0.1232 (0.1731)  loss_rpn_box_reg: 0.0210 (0.0215)  time: 20.7886  data: 20.4083  m

AttributeError: ignored

## Section 4: Test the model

In [ ]:
import matplotlib.patches as patches
from torchvision import transforms as torchtrans  

def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')

def apply_nms(orig_prediction, iou_thresh=0.3):
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [ ]:
#TODO: Pick an one image from the test set and print it's precidition as well as the bounding box using above helper functions.
# Load an image from the test set
img, target = test_dataset[0]

# Plot the image and ground truth bounding boxes
plot_img_bbox(torch_to_pil(img), target)

# Make a prediction using the trained model
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

# Apply non-maximum suppression to the prediction
prediction = apply_nms(prediction[0])

# Plot the image and predicted bounding boxes
plot_img_bbox(torch_to_pil(img), prediction)
